In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
  

data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Membagi kolom tanggal menjadi tahun dan bulan
df['tahun'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[0]))
df['bulan'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[1]))

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Memisahkan fitur dan target
X = df[['tahun', 'bulan']].values
y = df['jumlah_kecelakaan'].values

In [3]:
# Pisahkan data menjadi training dan testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Fungsi untuk menghitung MAPE menggunakan model SVR dengan parameter tertentu
def calculate_mape(X_train, y_train, X_val, y_val, params):
    C, epsilon, gamma = params
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred) *100
    return mape

# Inisialisasi populasi awal
def initialize_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        chromosome = []
        for param_range in parameter_ranges:
            param_value = np.random.uniform(param_range[0], param_range[1])
            chromosome.append(param_value)
        population.append(chromosome)
    return population

# Seleksi individu berdasarkan nilai fitness (MAPE)
def selection(population, X_train, y_train, X_val, y_val):
    selected_population = []
    for chromosome in population:
        mape = calculate_mape(X_train, y_train, X_val, y_val, chromosome)
        selected_population.append((chromosome, mape))
    selected_population.sort(key=lambda x: x[1])  # Sort berdasarkan MAPE terendah
    return selected_population[:len(population)//2]  # Pilih setengah individu terbaik

# Crossover: Menghasilkan keturunan dari pasangan individu terpilih
def crossover(selected_population):
    new_population = []
    ukuran_populasi = len(selected_population)
    
    for i in range(ukuran_populasi):
        parent1 = selected_population[i][0]
        
        # Tentukan parent2 dengan pemeriksaan batas
        if i % 2 == 0:
            if i - 1 >= 0:
                parent2 = selected_population[i-1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i-1 tidak valid
        else:
            if i + 1 < ukuran_populasi:
                parent2 = selected_population[i+1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i+1 tidak valid
        
        crossover_point = np.random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)
    
    return new_population

# Mutasi: Mengubah nilai gen secara acak pada populasi baru
def mutation(new_population, mutation_rate, parameter_ranges, sigma_range):
    for i in range(len(new_population)):
        if np.random.random() < mutation_rate:
            mutated_gene_index = np.random.randint(0, len(new_population[i]))
            sigma = np.random.uniform(sigma_range[0], sigma_range[1])
            new_value = new_population[i][mutated_gene_index] + np.random.normal(0, sigma)
            new_population[i][mutated_gene_index] = np.clip(new_value, parameter_ranges[mutated_gene_index][0], parameter_ranges[mutated_gene_index][1])
    return new_population

In [5]:
# Rentang nilai parameter optimal
parameter_ranges = [(0.01, 1), (0.001, 1), (1e-07, 0.01)]

# Inisialisasi populasi awal
population_size = 50
population = initialize_population(population_size, parameter_ranges)

# Iterasi algoritma genetika
crossover_rate = 1  # Tetapkan tingkat crossover tetap untuk eksperimen
mutation_rate = 0.0  # Tetapkan tingkat mutasi tetap untuk eksperimen
num_generations = 125  # Tetapkan jumlah generasi tetap untuk eksperimen
sigma_range = (0.001, 0.1)

for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    selected_population = selection(population, X_train, y_train, X_val, y_val)
    for i, (chromosome, mape) in enumerate(selected_population):
        print(f"Individu {i + 1}: C={chromosome[0]}, epsilon={chromosome[1]}, gamma={chromosome[2]}, MAPE={mape}")
    
    new_population = crossover(selected_population)
    new_population = mutation(new_population, mutation_rate, parameter_ranges, sigma_range)

    # Adjust the population size if necessary
    if len(new_population) < population_size:
        additional_individuals = initialize_population(population_size - len(new_population), parameter_ranges)
        new_population.extend(additional_individuals)
    elif len(new_population) > population_size:
        new_population = new_population[:population_size]
    
    # Make sure the new population has the correct size
    assert len(new_population) == population_size, "Ukuran populasi baru tidak sesuai"
    
    population = new_population

# Mengambil individu terbaik (dengan MAPE terendah) dari populasi terakhir
best_chromosome, best_mape = min(selected_population, key=lambda x: x[1])

# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='rbf', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nParameter terbaik:")
print(f"C: {best_chromosome[0]}")
print(f"Epsilon: {best_chromosome[1]}")
print(f"Gamma: {best_chromosome[2]}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu 1: C=0.7698233186983118, epsilon=0.04945561923169722, gamma=0.00269465078344822, MAPE=19.41574498243069
Individu 2: C=0.6062715747717373, epsilon=0.043298899280913294, gamma=0.007301837397878264, MAPE=19.606633009749967
Individu 3: C=0.6898037882469872, epsilon=0.7983560013731985, gamma=0.004664445651119158, MAPE=20.390900629871226
Individu 4: C=0.3580922549538881, epsilon=0.8479957103977513, gamma=0.009340483473715744, MAPE=20.390900629871226
Individu 5: C=0.6783878698989441, epsilon=0.6275879805156769, gamma=0.009787829555603556, MAPE=20.390900629871226
Individu 6: C=0.5963578105781363, epsilon=0.5822184235971803, gamma=0.005403683229319216, MAPE=20.390900629871226
Individu 7: C=0.24027706855828804, epsilon=0.5639120936962214, gamma=0.009851870553488889, MAPE=20.390900629871226
Individu 8: C=0.09834229510226163, epsilon=0.7191609833358337, gamma=0.009953199939906327, MAPE=20.390900629871226
Individu 9: C=0.9058327644232301, epsilon=0.8083004361927353, gamma=0.007

In [6]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[16.9895598 ]
 [17.70086464]
 [18.37486597]
 [18.98770004]
 [19.51726521]
 [19.94452075]
 [20.25460657]
 [20.43769318]
 [20.48949483]
 [20.41140865]
 [20.21027659]
 [19.89779978]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='poly', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [-1478779.98163262  -131819.59100762  -343719.10663262  2109698.03399238
  -833556.38007012  1661720.83086738  1728573.44805488    35383.63555488
 -1100800.60663262  2491198.47930488  2388337.20586738   572663.72149238]
MAPE: 4092832.584310634


In [8]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[-48799736.39387646]
 [ -4350043.50325146]
 [-11342727.51887646]
 [ 69620038.12174854]
 [-27507357.54231396]
 [ 54836790.41862353]
 [ 57042926.78581103]
 [  1167662.97331104]
 [-36326417.01887646]
 [ 82209552.81706104]
 [ 78815130.79362354]
 [ 18897905.80924854]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='linear', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.56947632 0.57380533 0.57813433 0.58246333 0.58679234 0.59112134
 0.59545035 0.59977936 0.60410836 0.60843736 0.61276637 0.61709537]
MAPE: 26.806405471655882


In [10]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 6.30577345]
 [ 7.6250653 ]
 [16.15278229]
 [18.5275075 ]
 [ 9.85196813]
 [21.01826833]
 [15.24517118]
 [19.43511846]
 [ 9.58810981]
 [21.8098436 ]
 [18.79136578]
 [19.96283495]]


In [11]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='sigmoid', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.3316345 0.3316345 0.3316345 0.3316345 0.3316345 0.3316345 0.3316345
 0.3316345 0.3316345 0.3316345 0.3316345 0.3316345]
MAPE: 0.5897912781957485


In [12]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]
 [13.94393863]]
